# Multi-tool Agent

You will modify the agent to add a _Text to Cypher_ retriever tool.

The Text to Cypher tool will allow the agent to create queries to retrieve more specific information such as facts and figures.

***

Load the environment variables, import the required Python modules, and create the retrieve financial documents tool.

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

from langchain.chat_models import init_chat_model
from langgraph.prebuilt import create_react_agent
from langchain_core.tools import tool
from langchain_neo4j import Neo4jGraph, Neo4jVector, GraphCypherQAChain
from langchain_openai import OpenAIEmbeddings
from langchain_core.prompts import PromptTemplate

# Initialize the LLM
model = init_chat_model("gpt-4o", model_provider="openai")

# Create the embedding model
embedding_model = OpenAIEmbeddings(model="text-embedding-ada-002")

# Connect to Neo4j
graph = Neo4jGraph(
    url=os.getenv("NEO4J_URI"),
    username=os.getenv("NEO4J_USERNAME"), 
    password=os.getenv("NEO4J_PASSWORD"),
)

# Define the retrieval query
retrieval_query = """
MATCH (node)-[:FROM_DOCUMENT]-(doc:Document)-[:FILED]-(company:Company)
RETURN 
    node.text as text,
    score,
    {
        company: company.name,
        risks: [ (company:Company)-[:FACES_RISK]->(risk:RiskFactor) | risk.name ]
    } AS metadata
ORDER BY score DESC
"""

# Create Vector
chunk_vector = Neo4jVector.from_existing_index(
    embedding_model,
    graph=graph,
    index_name="chunkEmbeddings",
    embedding_node_property="embedding",
    text_node_property="text",
    retrieval_query=retrieval_query,
)

# Define functions for each tool in the agent

@tool("Get-graph-database-schema")
def get_schema():
    """Get the schema of the graph database."""
    context = graph.schema
    return context

@tool("Retrieve-financial-documents")
def retrieve_docs(query: str):
    """Find details about companies in their financial documents."""
    # Use the vector to find relevant documents
    context = chunk_vector.similarity_search(
        query, 
        k=3,
    )
    return context

The Text to Cypher tool can use a separate LLM to generate the Cypher. This is useful as different models and settings are more effective at generating Cypher.

***

Create a `cypher_model` with a `temperature` of `0.0`.

In [2]:
# Create a separate model for Cypher generation
cypher_model = init_chat_model(
    "gpt-4o", 
    model_provider="openai",
    temperature=0.0
)

The Text to Cypher tool requires a prompt which instructs the LLM on how to generate the Cypher.

***

Create a `cypher_template` which accepts the graph `schema` and the user's `question`.

In [3]:
# Create a cypher generation prompt
cypher_template = """Task:Generate Cypher statement to query a graph database.
Instructions:
Use only the provided relationship types and properties in the schema.
Do not use any other relationship types or properties that are not provided.

Use `WHERE tolower(node.name) CONTAINS toLower('name')` to filter nodes by name.

Schema:
{schema}

Note: Do not include any explanations or apologies in your responses.
Do not respond to any questions that might ask anything else than for you to construct a Cypher statement.
Do not include any text except the generated Cypher statement.

The question is:
{question}"""

The prompt can include specific instructions on how to generate Cypher, for example, this instruction:

> Use `WHERE tolower(node.name) CONTAINS toLower('name')` to filter nodes by name.

... tells the LLM to use case insensitive and wild card matching when searching by company name.

***

Create a `cypher_prompt` using the template you just created.

In [4]:
cypher_prompt = PromptTemplate(
    input_variables=["schema", "question"], 
    template=cypher_template
)

Create a Cypher QA (question/answer) chain using the `cypher_model` and `cypher_prompt`.

In [5]:
# Create the Cypher QA chain
cypher_qa = GraphCypherQAChain.from_llm(
    graph=graph, 
    llm=model,
    cypher_llm=cypher_model,
    cypher_prompt=cypher_prompt,
    allow_dangerous_requests=True,
    return_direct=True,
    verbose=True
)

The `return_direct` flag instructs the `cypher_qa` chain to return just the output of the Cypher query. 

Setting `verbose` will output the generated Cypher so you can see the results.

<div class="alert alert-block alert-warning">
<b>Allow Dangerous Requests</b><br/>
You are trusting the generation of Cypher to the LLM.
It may generate invalid Cypher queries that could corrupt data in the graph or provide access to sensitive information.

You have to opt-in to this risk by setting the `allow_dangerous_requests` flag to `True`.

In a production environment, you should ensure that access to data is limited, and sufficient security is in place to prevent malicious queries. 
</div>

***

Create a new tool to `Query-database` that uses the `cypher_qa` chain.

In [6]:
@tool("Query-database")
def query_database(query: str):
    """Get answers to specific questions about companies, risks, and financial metrics."""
    context = cypher_qa.invoke(
        {"query": query}
    )
    return {"context": context}

Create the agent `tools` and the `agent`.

In [7]:
# Add the tools to the agent
tools = [get_schema, retrieve_docs, query_database]

agent = create_react_agent(
    model, 
    tools
)

Create a query, run the agent, and stream the results.

In [9]:
# Run the application
query = "What products does Microsoft mention in its financial documents?"

for step in agent.stream(
    {
        "messages": [{"role": "user", "content": query}]
    },
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What products does Microsoft mention in its financial documents?
================================== Ai Message ==================================
Tool Calls:
  Retrieve-financial-documents (call_6UNAnQ9GudJVqNFPUcSb05ac)
 Call ID: call_6UNAnQ9GudJVqNFPUcSb05ac
  Args:
    query: Microsoft products mentioned in financial documents
================================= Tool Message =================================
Name: Retrieve-financial-documents

[Document(metadata={'risks': ['Racial Injustice', 'Inequity', 'racial equity and fairness in the justice system', 'affordable broadband, devices, and digital literacy', 'Piracy', 'Intense competition across all markets', 'Competition in the technology sector', 'Competition among platform-based ecosystems', 'Competition with business models', 'Cost of converting ideas to software products', 'Investment in AI development', 'Open source competition', 'Decreased sales 

Depending what question you ask the agent will use different tools to respond to the question.

***

Modify the question and observe how the agent changes tool, or even runs multiple tools, to gather the context it requires to answer the question.

Try these examples:

* What are the main risk factors mentioned in the documents?
* How does the graph model relate to financial documents and risk factors?
* What products does Microsoft mention in its financial documents?
* Summarize Apple's risk factors and how they relate to other companies
* How many risk facts does Apple face and what are the top ones?

***

[View the complete code](solutions/02_03_text2cypher_agent.py)